In [1]:
import torch

x = torch.zeros((3,4))


In [4]:
x[0][-5:]

tensor([0., 0., 0., 0.])

In [5]:
d = {'a': [[1,2,3], [3,4,5]]}
d

{'a': [[1, 2, 3], [3, 4, 5]]}

In [23]:
d['a'][0] = [1,2]
d

{'a': [[1, 2], [3, 4, 5]]}

In [22]:
my_dict = {'a': 1, 'b': 2, 'c': 3}
value = next(iter(my_dict.values()))
print(value)  # 输出 1 或 2 或 3

1


In [ ]:
lengths = [3, 32, 15, 7]
num_tokens_to_remove = 20
while num_tokens_to_remove:
    

In [124]:
import numpy as np
import time

num = 50000

start = time.time()

for _ in range(num):
    lengths = [312, 267]
    num_tokens_to_remove = 50
    lengths_waiting_to_trunc = [lengths[i] for i in [0,1]]
    for _ in range(num_tokens_to_remove):
        index_max = lengths_waiting_to_trunc.index(max(lengths_waiting_to_trunc))
        lengths_waiting_to_trunc[index_max] -= 1
end = time.time()

print(end-start)
print(lengths_waiting_to_trunc)


0.9324193000793457
[264, 265]


In [121]:
import numpy as np
import time


def argmax(l: list):
    max_value_idx = 0
    max_value = l[0]
    idx = 1
    while idx<len(l):
        if max_value<l[idx]:
            max_value, max_value_idx = l[idx], idx
        idx += 1
    return max_value_idx


lengths = [312, 267]
x = 50
start = time.time()
for _ in range(num):
    lengths = [312, 267]*30
    for _ in range(x):
        max_index = argmax(lengths)
        lengths[max_index] -= 1
end = time.time()

print(end-start)
print(lengths)


7.97226357460022
[310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 310, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267, 311, 267]


In [113]:
import numpy as np
import time

lengths = [312, 267]
x = 50


start = time.time()
for _ in range(num):
    lengths = [312, 267]*3
    for _ in range(x):
        lengths = np.array(lengths)
        max_index = np.argmax(lengths)
        lengths[max_index] -= 1
end = time.time()

print(end-start)
print(lengths)

7.818470001220703
[295 267 295 267 296 267]


In [129]:
def __truncate(tokenized_dict: dict[str, list[list]], waiting_to_trunc_idxs: list, num_tokens_to_remove: int) -> None:
    lengths = [len(value_part) for value_part in tokenized_dict["input_ids"]]
    lengths_waiting_to_trunc = [lengths[i] for i in waiting_to_trunc_idxs]
    for _ in range(num_tokens_to_remove):
        index_max = lengths_waiting_to_trunc.index(max(lengths_waiting_to_trunc))
        lengths_waiting_to_trunc[index_max] -= 1
    print(lengths_waiting_to_trunc)
    for i, j in zip(waiting_to_trunc_idxs, range(len(lengths_waiting_to_trunc))):
        lengths[i] = lengths_waiting_to_trunc[j]
    for value in tokenized_dict.values():
        for i in waiting_to_trunc_idxs:
            value[i] = value[i][: lengths[i]]

cur_dict = {'input_ids':[[1,2,3,4,5], [6,7,8,9,0]], 'ttt': [[5,4,3,2,1], [6,7,8,9,0]]}
__truncate(cur_dict, [0,1], 4)
cur_dict
cur_dict = {key: sum(value, []) for key, value in cur_dict.items()}
cur_dict

[3, 3]


{'input_ids': [1, 2, 3, 6, 7, 8], 'ttt': [5, 4, 3, 6, 7, 8]}

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [57]:
inputs = tokenizer('hello, how are you. hahahha', add_special_tokens=False)
inputs

{'input_ids': [7592, 1010, 2129, 2024, 2017, 1012, 5292, 3270, 23644, 2050], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [59]:
inputs = dict(inputs)

In [61]:
type(inputs)
inputs

{'input_ids': [7592, 1010, 2129, 2024, 2017, 1012, 5292, 3270, 23644, 2050],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [63]:
inputs = tokenizer.pad(inputs)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from toolkit.nlp.dataset import TextDataset
from transformers import AutoTokenizer
import jsonlines

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def myfun(file_path, tokenizer, model_type, **kargs):
    special_tokens_map = tokenizer.special_tokens_map
    BOS = special_tokens_map["bos_token"] if "bos_token" in special_tokens_map.keys() else None
    EOS = special_tokens_map["eos_token"] if "eos_token" in special_tokens_map.keys() else None
    SEP = special_tokens_map["sep_token"] if "sep_token" in special_tokens_map.keys() else None
    MASK = special_tokens_map["mask_token"] if "mask_token" in special_tokens_map.keys() else None
    CLS = special_tokens_map["cls_token"] if "cls_token" in special_tokens_map.keys() else None
    sep_num = 1

    with jsonlines.open(file_path, "r") as jlReader:
        dict_objs = list(jlReader)
        if isinstance(dict_objs[0], str):
            dict_objs = dict_objs[1:]

    input_texts = []
    labels = []
    for dict_obj in dict_objs:
        # input_texts.append(f"{BOS}{dict_obj['question1']}{SEP*sep_num}{dict_obj['question2']}{EOS}")
        input_texts.append(((False, CLS), (True, dict_obj["question1"]), (False, SEP), (True, dict_obj["question2"]), (False, SEP)))
        labels.append([dict_obj["label"]])
        # labels.append(((False, CLS), (True, dict_obj["question1"])))
        # labels.append(dict_obj["question1"])
    return input_texts, labels


dataset = TextDataset(
    "test/data.jsonl",
    "bert-base-uncased",
    tokenizer,
    input_text_format_func=myfun,
    padding_side="left",
    max_length_input=256,
    max_length_label=256,
    is_train=False,
)

2023-06-26 10:46:29,667 -  WARNING > toolkit.nlp.dataset: model input include 'token_type_ids'. There is a bug causing all the token_type_ids to be zeros


Tokenize input texts:   0%|          | 0/32 [00:00<?, ?it/s]

In [275]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [270]:
print(dataset[0])

{'model_input': {'input_ids': tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         101, 2339, 2024, 3060, 1011, 4841, 2061, 3376, 1029,  102, 2339, 2024,
        6696, 2015, 2061, 3376, 1029,  102]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])}, 'first_not_pad_index_input': tensor(36), 'labels': tensor([0], dtype=torch.int32)}


In [271]:
print(dataset.max_length_inputs)
print(dataset.max_length_labels)
print(tokenizer.decode(dataset[1]['model_input']['input_ids']))

54
1
[CLS] i want to pursue phd in computer science about social network, what is the open problem in social networks? [SEP] i handle social media for a non - profit. should i start going to social media networking events? are there any good ones in the bay area? [SEP]


In [272]:
from torch.utils.data import DataLoader, default_collate

dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=False, 
                        collate_fn=dataset.collate_fn_padding_right if dataset.padding_side=='right' else dataset.collate_fn_padding_left, 
                        pin_memory=True)

for batch in dataloader:
    print(batch['input_ids'].shape)
    # print(batch['labels'].shape)
    # print(batch['input_ids'].is_pinned())
    # print(batch)
    # break

print(batch)

torch.Size([4, 54])
torch.Size([4, 39])
torch.Size([4, 46])
torch.Size([4, 33])
torch.Size([4, 36])
torch.Size([4, 48])
torch.Size([4, 35])
torch.Size([4, 50])
{'labels': tensor([[0],
        [0],
        [1],
        [0]], dtype=torch.int32), 'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,   101,  2054,  2024,
          1996,  5918,  2000,  2468,  2343,  1999,  1996,  2142,  2163,  1998,
          2129,  2024,  1996,  5918,  2367,  1999, 11959,  1029,   102,  2054,
          2024,  1996,  5918,  2000,  2468,  2343,  1999,  1996,  2142,  2163,
          1998,  2129,  2024,  1996,  5918,  2367,  1999,  2605,  1029,   102],
        [  101,  1000,  2040,  2003,  1996,  4205,  1996,  4581,  2000, 12373,
          1011, 17691,  1005,  1055,  1000,  1000,  4205,  1005,  1055,  2299,
          1000,  1000,  2024,  2517,  2055,  1029,  1000,   102,  1000,  2029,
         12373,  1011, 17691,  2316,  2266,  2626,  1996,  4581,  2000,  1000,
          1000,  4205,  1005,  

In [234]:
from torch.utils.data import DataLoader, default_collate

default_collate([{'a': torch.tensor([1,2,3]), 'b': torch.tensor([4,5,6]), 'c':'h' 'h'}, {'a': torch.tensor([1,2,3]), 'b': torch.tensor([4,5,6]), 'c':'h' 'h'}])

{'a': tensor([[1, 2, 3],
         [1, 2, 3]]),
 'b': tensor([[4, 5, 6],
         [4, 5, 6]]),
 'c': ['hh', 'hh']}

In [ ]:
from transformers import PretrainedConfig, BertConfig

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from toolkit import Config
# Config.attribute_alias_map = {'return_dict': 'retrundict'}
t = Config(return_dict=True)

print(f"self.__dict__: {t.__dict__}")
print(t.to_dict())
print(t.to_diff_dict())
t.return_dict

self.__dict__: {'model_type': '', 'model_name': '', '_name_or_path': '', 'return_dict': True}
{'model_type': '', 'model_name': '', '_name_or_path': '', 'return_dict': True}
{'return_dict': True}


True

In [4]:
from toolkit.nlp import TrainConfig

kwargs = {
    "dataset":'QQP',
    "early_stop_metric": 'acc',
    "model_type": "bert-base-uncase",
    "model_name": "baseline",
    "epochs": 5,
    "batch_size": 32,
    "learning_rate": 3e-5,
    "a_unknow": 'aaa'
}
tt = TrainConfig(**kwargs)
tt.save_pretrained('config')
print(tt.model_type)
tt

2023-06-30 07:33:26,312 -  INFO > toolkit.configuration: Configuration saved in config/train_config.json
bert-base-uncase


TrainConfig {
  "a_unknow": "aaa",
  "accumulate_step": 1,
  "adam_epsilon": 1e-08,
  "batch_size": 32,
  "dataset": "QQP",
  "early_stop": false,
  "early_stop_metric": "acc",
  "epochs": 5,
  "fp16": false,
  "learning_rate": 3e-05,
  "max_length_input": null,
  "max_length_label": null,
  "model_name": "baseline",
  "model_type": "bert-base-uncase",
  "problem_type": null,
  "seed": 0,
  "test_in_epoch": false,
  "warmup": false,
  "warmup_ratio": -1,
  "weight_decay": 0.01
}

In [5]:
tt_load = TrainConfig.from_pretrained('train_config.json')
print(tt==tt_load)
print(tt_load)
tt.max_length_input is None

2023-06-30 07:33:30,424 -  INFO > toolkit.configuration: loading configuration file config/train_config.json
True
TrainConfig {
  "a_unknow": "aaa",
  "accumulate_step": 1,
  "adam_epsilon": 1e-08,
  "batch_size": 32,
  "dataset": "QQP",
  "early_stop": false,
  "early_stop_metric": "acc",
  "epochs": 5,
  "fp16": false,
  "learning_rate": 3e-05,
  "max_length_input": null,
  "max_length_label": null,
  "model_name": "baseline",
  "model_type": "bert-base-uncase",
  "problem_type": null,
  "seed": 0,
  "test_in_epoch": false,
  "warmup": false,
  "warmup_ratio": -1,
  "weight_decay": 0.01
}



True

In [29]:
class TrainConfig(Config):
    model_type='bert'
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.train_return_dict = False
        self.ttt = 1

st = TrainConfig()
print(st.model_type)
st.ttt=2
print(st.to_dict())
print(st.to_diff_dict())

bert
{'_name_or_path': '', 'train_return_dict': False, 'ttt': 2, 'model_type': 'bert'}
base:        {'_name_or_path': '', 'model_type': ''}
class specific: {'_name_or_path': '', 'train_return_dict': False, 'ttt': 1, 'model_type': 'bert'}
{'train_return_dict': False, 'ttt': 2, 'model_type': 'bert'}


In [32]:
TrainConfig.from_pretrained('')

2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.
2023-06-29 14:21:40,606 -  WARNING > toolkit.configuration: bert. This is not supported for all configurations of models and can yield errors.

FileNotFoundError: [Errno 2] No such file or directory: 'config'

In [39]:
t.attribute_alias_map

attribute_alias_map


{'return_dict': 'retrundict'}

In [40]:
t.return_dict

return_dict
retrundict


True

In [41]:
t.retrundict

retrundict


True

In [55]:
import os


class Path(os.PathLike):
    def __init__(self, path):
        self.__path = path

    def __fspath__(self):
        return self.__path.replace("/", os.path.sep)

    
path = Path('/home/jjwang/')
print(path.__fspath__())

/home/jjwang/


In [67]:
from pathlib import Path

path = Path('./test/data.jsonl')


In [139]:
import re
text = """
复杂推理是指理解和利用支持证据或逻辑得出结论或做出决定的能力[51,52]。根据推理过程中涉及的逻辑和证据的类型，将现有的评估任务分为知识推理、符号推理和数学推理3大类。

"""

text = re.sub(r'。', '。\n', text)
text = re.sub(r'\(', '（', text)
text = re.sub(r'\)', '）', text)
text = re.sub(r'llm', 'LLM', text)
text = re.sub(r'plm', 'PLM', text)
print(text)



复杂推理是指理解和利用支持证据或逻辑得出结论或做出决定的能力[51,52]。
根据推理过程中涉及的逻辑和证据的类型，将现有的评估任务分为知识推理、符号推理和数学推理3大类。





In [42]:
from pathlib import Path
p = Path('setup.cfg')

In [43]:
p.exists()

True

In [44]:
p.is_file()

True

In [45]:
p.is_dir()

False

In [46]:
'config'/p

PosixPath('config/setup.cfg')